### Цель:
разработать систему, которая находит в предложении название фильма. 
Если название найдено, формируется итоговое предложение по шаблону. 
В остальных случаях итоговое предложение не отличается от полученного на вход.

### Входные данные:
`предложение` с вопросом о фильме & необработанный `список` из доступных названий фильмов
### 2 варианта Выходных данных:
1. Последовательность вида "Рецензия на фильм название_фильма:", 
если в предложении обнаружено название фильма
2. Оригинальное предложение, если название фильма не распознано

#### `пример 1:`
* `Вход:` "Как тебе фильм Матрица?"
* `Выход:` "Рецензия на фильм Матрица:"

#### `пример 2:`
* `Вход:` "Тебе понравилась игра Киану Ривза?"
* `Выход:` "Тебе понравилась игра Киану Ривза?"

------------------------------------------------

### Возможный алгоритм

1. Создаем словарь из токенизированных названий фильмов, предварительно очистив названия от символьного мусора "/ : ; ," и тд.
#### `Пример:` {'Остров проклятых': ['остров','проклятых'], 'Матрица: Воскрешение': ['матрица','воскрешение']}

2. Создаем копию предложения и все преобразования делаем на копии
3. Разбиваем полученное предложение на токены
4. Применяем lowercase для всех букв
5. Проводим лемматизацию каждого токена
6. Обрезаем каждый токен до 2/3 от его изначальной длины 
#### `Пример:` 'собрание' -> 'собра', т.к. 8 * 2/3 = 5

7. Проводим сравнение последовательностей из предложения со словарем названий.
В токенизированном тексте последовательно проверяется каждый токен. 

(а) если последовательность из токенов предложения совпадает с 1 из последовательностью названия фильма

-> в предложении упоминается название фильма 

-> на выходе система возвращает шаблон "Рецензия на фильм Оригинальное название фильма:"
#### `Пример:`
* ориг. предложение: 'Поделись мнением о фильме Матрица: Воскрешение'
* токениз. предложение: ['поде', 'мнен', 'о', 'филь', 'матр', 'воскреш']
* словарь названий: {..., 'Остров проклятых': ['остров','проклятых'], 'Матрица: Воскрешение': ['матрица','воскрешение'], ...}


* выход: 'Рецензия на фильм Матрица: Воскрешение:' 
(т.к. последовательность 'матр', 'воскреш' входит в последовательность 'матрица','воскрешение')

(б) если ни одно совпадение не обнаружено 
-> на выходе система возвращает оригинальное предложение
#### `Пример:`
* ориг. предложение: 'Почему Николас Кейдж снимается в откровенном треше?'


* выход: 'Почему Николас Кейдж снимается в откровенном треше?'


In [1]:
#!pip install numpy pandas
#!pip install nltk sklearn  scipy
#!pip3 install pymystem3
#!pip3 install git+https://github.com/Koziev/rutokenizer

In [2]:
import numpy as np 
import pandas as pd
import rutokenizer
import re

import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import RegexpTokenizer


In [3]:
films = pd.read_csv("filtered_films.csv")
films

,Unnamed: 0,filtered_film_names
0,0,Помни
1,1,Форрест Гамп
2,2,Зеленая миля
3,3,Крестный отец
4,4,Король Лев
...,...,...
171,171,Мистер Сталь
172,172,Побег невозможен
173,173,Милашка в розовом
174,174,Призрак и миссис Мьюр


In [4]:
#название фильмы в ловер, лемма, токеннизация 
test_films = films.copy()
test_films = test_films.drop(columns = ['Unnamed: 0'], axis = 1)
test_films = test_films['filtered_film_names'].str.lower()

test_films[20]

'12 обезьян'

In [5]:
def regular_cleaning(test_films):
    temp = 0
    for row in test_films.index:
        temp = re.sub(r"[0-9]{1,}-[а-я]{1,}", ",", test_films[row])
        temp = re.sub(r"  ", "", temp)
        temp = re.sub(r"[-—.?:;!»«/\)(#$%^&*№%']", " ", temp)
        temp = re.sub(r"[ ]{1,}", ",", temp)
        if temp.endswith(" "):
            temp = temp[:-1]
        if temp.startswith(" "):
            temp = temp[1:]
        test_films[row] = temp
    return test_films



In [6]:
regular_cleaning(test_films)

0                      помни
1               форрест,гамп
2               зеленая,миля
3              крестный,отец
4                 король,лев
               ...          
171             мистер,сталь
172         побег,невозможен
173        милашка,в,розовом
174    призрак,и,миссис,мьюр
175       это,старое,чувство
Name: filtered_film_names, Length: 176, dtype: object

In [7]:
test_films[20]

'12,обезьян'

In [8]:
test_films[30]

'звёздные,войны,эпизод,6,возвращение,джедая'

In [9]:
film_tokens = test_films.copy()

In [17]:
def clean_tokens(text):
    regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
    text = re.sub(regular, ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text
  
cleaned_text = []
tokens = []
 
for text in test_films:
    text = clean_tokens(text)
    cleaned_text.append(text)

    text = word_tokenize(text)
    tokens.append(text)
    
film_tokens['cleaned_films'] = cleaned_text
film_tokens['tokens_films'] = tokens

In [18]:
tokens

[['помни'],
 ['форрест', 'гамп'],
 ['зеленая', 'миля'],
 ['крестный', 'отец'],
 ['король', 'лев'],
 ['американская', 'история', 'x'],
 ['криминальное', 'чтиво'],
 ['гладиатор'],
 ['искусственный', 'разум'],
 ['дракула'],
 ['анастасия'],
 ['гражданин', 'кейн'],
 ['мертвец'],
 ['привидение'],
 ['мосты', 'округа', 'мэдисон'],
 ['большой', 'лебовски'],
 ['грязные', 'танцы'],
 ['звонок'],
 ['амадей'],
 ['казино'],
 ['12', 'обезьян'],
 ['скала'],
 ['магнолия'],
 ['чужой', '3'],
 ['50', 'первых', 'поцелуев'],
 ['огни', 'большого', 'города'],
 ['битлджюс'],
 ['фарго'],
 ['кейт', 'и', 'лео'],
 ['простая', 'история'],
 ['звёздные', 'войны', 'эпизод', '6', 'возвращение', 'джедая'],
 ['рекрут'],
 ['смерть', 'ей', 'к', 'лицу'],
 ['вспомнить', 'всё'],
 ['убить', 'пересмешника'],
 ['аладдин'],
 ['история', 'рыцаря'],
 ['чего', 'хотят', 'женщины'],
 ['королевская', 'битва'],
 ['мумия', 'возвращается'],
 ['трудная', 'мишень'],
 ['тельма', 'и', 'луиза'],
 ['послание', 'в', 'бутылке'],
 ['бездна'],
 ['лж

In [12]:
from pymystem3 import Mystem
m = Mystem()

def lemmatize_str(sent, myStemObj):
    lemmat_list = []
    try:
        lemmas = myStemObj.lemmatize(sent)
        for i in lemmas:
            if i.isalpha():
                lemmat_list.append(i)
        return lemmat_list
    except BrokenPipeError:
        print(sent)

In [13]:
lemma_films_1 = test_films.apply(lambda x: lemmatize_str(x, m))
lemma_films_1

0                       [помнить]
1                 [форрест, гамп]
2                 [зеленый, миля]
3                [крестный, отец]
4                   [король, лев]
                  ...            
171               [мистер, сталь]
172          [побег, невозможный]
173         [милашка, в, розовый]
174    [призрак, и, миссис, мьюр]
175       [этот, старый, чувство]
Name: filtered_film_names, Length: 176, dtype: object

In [20]:
lemma_films_1[30]

['звездный', 'война', 'эпизод', 'возвращение', 'джедай']

In [15]:
# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
# w_tokenizer.tokenize(lemma_films_1[17])


In [16]:
# def clean_tokens(text):
#     regular = r'[\*+\#+\№\"\-+\+\=+\?+\&\^\.+\;\,+\>+\(\)\/+\:\\+]'
#     text = re.sub(regular, '', text)
#     text = re.sub(r'\s+', '', text)
#     return text
  
# cleaned_text = []
# tokens = []
 
# # для каждого сообщения text из столбца data['Message']
# for text in lemma_films_1.index:
#     text = clean_tokens(text)
#     cleaned_text.append(text)
#     #разбиваем текст на токены с сохраняем результат в списке tokens
#     text = word_tokenize(lemma_films_1[text])
#     tokens.append(lemma_films_1[text])
# film_tokens['cleaned_films'] = cleaned_text
# # Сохраняем разбитые на токены сообщения в новой колонке 'Tokens_msg'
# film_tokens['tokens_films'] = tokens